In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    
import keras
from keras import backend as K
from keras import regularizers
from keras.models import Model, Sequential, load_model
from keras.activations import relu, softmax
from keras.layers import Input, BatchNormalization, Activation, Flatten, Dense, Dropout, \
GlobalMaxPooling2D, GlobalAveragePooling2D, Conv2D, MaxPooling2D, AveragePooling2D, Add
from get_net import get_resnet50, get_VGG16, get_efficientnetB0, get_InceptionResNetV2, get_MobileNetV2, get_Xception, get_NASNetLarge,get_InceptionV3
from block import block
from image_generator import get_generator
from inceptionv4 import *
from keras.optimizers import SGD, Adam, RMSprop
from keras.regularizers import l2
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import os

filepath="checkpoint/model_{epoch:03d}-{val_loss:.5f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',verbose=1, 
                            save_best_only=True)

tbCallBack = TensorBoard(log_dir='logs',
                 histogram_freq=0,  
#                  batch_size=32,     
                 write_images=True,
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)
# 資料路徑
DATASET_PATH  = './'

# 影像大小
IMAGE_SIZE = (224,224,3)

# 影像類別數
NUM_CLASSES = 5

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 0

# Epoch 數
NUM_EPOCHS = 100

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model.h5'

# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=0,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=(IMAGE_SIZE[0],IMAGE_SIZE[1]),
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/test',
                                                  target_size=(IMAGE_SIZE[0],IMAGE_SIZE[1]),
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))




#net_final = create_model(NUM_CLASSES, 0.2, 'imagenet', False,IMAGE_SIZE)
net_final = get_resnet50(None,IMAGE_SIZE,FREEZE_LAYERS,NUM_CLASSES,0)
#net_final = define_model()
#net_final = load_model("checkpoint/model_002-0.42074.hdf5")
print("no. of layers: "+str(len(net_final.layers)))

print(net_final.summary())
opt = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
#opt = RMSprop(lr=1e-5)
net_final.compile(optimizer=opt,
                  loss='categorical_crossentropy', metrics=['accuracy'])
# 訓練模型

net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks = [checkpoint,tbCallBack])

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

Using TensorFlow backend.


Found 18000 images belonging to 5 classes.
Found 2000 images belonging to 5 classes.
Class #0 = drawings
Class #1 = hentai
Class #2 = neutral
Class #3 = porn
Class #4 = sexy
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



c:\users\jk\anaconda3\envs\face\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


no. of layers: 178
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 166, 166, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 80, 80, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 80, 80, 64)   256         conv1[0][0]                      
_________________________________________________________________________